In [1]:
import torch
from supervoice_gpt import SupervoiceGPT, Tokenizer, config

In [2]:
# Model
tokenizer = Tokenizer(config, "tokenizer_text.model")
model = SupervoiceGPT(config)
checkpoint = torch.load(f'./output/pitch-2.pt', map_location="cpu")
model.load_state_dict(checkpoint['model'])
model.eval()
print('ok')

ok


In [3]:
tokens = model.generate("Clicking on any sample will automatically pause the other samples :)", tokenizer, max_new_tokens = 64, top_k = 6)
print(tokens)

[('c', 11, 0), ('ʎ', 3, 0), ('ɪ', 3, 0), ('c', 8, 0), ('ɪ', 5, 0), ('ŋ', 6, 0), ('•', 0, 0), ('ɒ', 6, 0), ('n', 6, 0), ('•', 0, 0), ('ɛ', 9, 0), ('ɲ', 6, 0), ('i', 6, 175), ('•', 0, 0), ('s', 13, 175), ('æ', 9, 177), ('m', 7, 174), ('p', 6, 0), ('ə', 3, 0), ('ɫ', 7, 172), ('•', 0, 0), ('w', 5, 171), ('ɪ', 3, 0), ('ɫ', 4, 0), ('•', 0, 0), ('ɒ', 14, 0), ('t', 3, 168), ('ə', 4, 170), ('m', 7, 0), ('æ', 10, 173), ('tʲ', 3, 173), ('ɪ', 4, 174), ('c', 11, 175), ('ʎ', 3, 176), ('i', 4, 176), ('•', 0, 0), ('pʰ', 13, 177), ('ɑː', 16, 174), ('z', 9, 0), ('•', 0, 0), ('d̪', 3, 0), ('iː', 3, 0), ('•', 0, 0), ('ɐ', 6, 0), ('ð', 3, 171), ('ɚ', 5, 170), ('•', 0, 0), ('s', 11, 0), ('æ', 6, 0), ('m', 5, 0), ('p', 6, 0), ('ə', 3, 0), ('ɫ', 5, 0), ('z', 12, 0), ('•', 0, 0), ('aj', 13, 0), ('ɹ', 17, 0)]


In [4]:
# input = "Hey, you?"
# ctx_tokens = ['ç', 'ɪ']
# ctx_durations = [8, 10]
# probs, p = model.predict_next(input, ctx_tokens, ctx_durations, tokenizer)
# for i in range(len(probs)):
#     print(p[i] + ": "+ str(probs[i]))
